In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [16]:
db_con = sqlite3.connect('data/database.sqlite')
cursor = db_con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
print(cursor.fetchall())

[('Reviews',)]


In [17]:
pos_neg_reviews = pd.read_sql_query('SELECT * FROM Reviews WHERE Score !=3 LIMIT 10000', db_con)

In [18]:
def assignPositive(x):
    if(x>3):
        return 1
    return 0

In [19]:
colScore = pos_neg_reviews['Score']
posNeg = colScore.map(assignPositive)
pos_neg_reviews['Score'] = posNeg
pos_neg_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
Id                        10000 non-null int64
ProductId                 10000 non-null object
UserId                    10000 non-null object
ProfileName               10000 non-null object
HelpfulnessNumerator      10000 non-null int64
HelpfulnessDenominator    10000 non-null int64
Score                     10000 non-null int64
Time                      10000 non-null int64
Summary                   10000 non-null object
Text                      10000 non-null object
dtypes: int64(5), object(5)
memory usage: 781.4+ KB


In [22]:
def dataCleaning(df):
    
    #Removing duplicates
    df.drop_duplicates(subset={'UserId','ProfileName','Time','Text'}, keep='first', inplace=True)
    
    #HelpfulnessDenominator is the sum of helpful(HelpfulnessNumerator) and no opinion
    df = df[df.HelpfulnessNumerator <= df.HelpfulnessDenominator]
    return df

In [23]:
readyData = dataCleaning(pos_neg_reviews)
readyData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9564 entries, 0 to 9999
Data columns (total 10 columns):
Id                        9564 non-null int64
ProductId                 9564 non-null object
UserId                    9564 non-null object
ProfileName               9564 non-null object
HelpfulnessNumerator      9564 non-null int64
HelpfulnessDenominator    9564 non-null int64
Score                     9564 non-null int64
Time                      9564 non-null int64
Summary                   9564 non-null object
Text                      9564 non-null object
dtypes: int64(5), object(5)
memory usage: 821.9+ KB


In [10]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def expandContractions(text):
    words = text.split(' ')
    expandedText = ''
    for word in words:
        if(word in contractions):
            expandedText += contractions[word].split('/')[0] #Always considering 1st form of verb for list
        else:
            expandedText += word
        expandedText += ' '
    return expandedText

In [11]:
#https://lxml.de/
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/#
#https://pypi.org/project/tqdm/2.2.3/

from bs4 import BeautifulSoup
import re
from tqdm import tqdm #show a progress meter - just wrap any iterable 

def textCleaning(text):
    
    #Remove Hyperlinks
    # Regex: r'\S+', # sequence of non-whitespace
    text = re.sub(r'http\S+', '', text)
    
    #Remove all tags
    text = BeautifulSoup(text, 'lmxl').get_text()
    
    #Expand texts
    text = expandContractions(text)
    
    #Remove all words containing numbers
    #https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number
    text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    
    #Remove special characters
    text = re.sub('[^A-Za-z]+', ' ', text)


Yooo she has  done it 
